In [1]:
# import keras 

from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPooling2D, Convolution2D, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

Using TensorFlow backend.


In [2]:
# Prepare train and test data, using keras image generator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set target size and batch size,  number of classes

size=(128,128)
batch=100
num_class=3

In [3]:
# Process image data from directory

training_set = train_datagen.flow_from_directory('data/training_set',
            target_size=size,
            batch_size=batch,
            class_mode='binary')

test_set= test_datagen.flow_from_directory('data/test_set',
        target_size=size,
        batch_size=batch,
        class_mode='binary')

In [4]:
# Basic convolutional neural network structure

# Define a CNN classifier network composed of a sequence of layers

model = Sequential()

# The input layer accepts an image and applies a convolution that uses 32 3x3 filters and a rectified linear unit activation function
model.add(Convolution2D(32,3,3, input_shape=(128,128,3), activation='relu'))
# Next add a max pooling layer with a 2*2 patch
model.add(MaxPooling2D(pool_size=(2, 2)))


# Add more layers, including dropout layer that prevent over-fitting
model.add(Convolution2D(32,3,3, activation='relu'))
model.add(Dropout(0.25))

model.add(Convolution2D(64,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


# Flatten the feature maps and generate an output layer
num_classes=3
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model for categorical (multi-class) classification
opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
dropout_3 (Dropout)          (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 29, 29, 64)        0         
__________

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., activation="relu")`
  
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  from ipykernel import kernelapp as app


In [ ]:
# Train CNN model
# steps per epoch=train samples//batch size

num_epochs=30
history = model.fit_generator(
        training_set,
        steps_per_epoch=500,
        epochs=num_epochs,
        validation_data=test_set,
        validation_steps=250)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# from IPython.display import Inline
# plot the history of metrics to evaluate the model

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model evaluation')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Save the model

model.save('object_detection_model')